In [2]:
import re
from dateutil import parser

def extract_information(email_text):
    state_pattern = r'in (\w+) from'
    date_range_pattern = r'from ([\d/]+) through ([\d/]+)'
    columns_pattern = r'columns:(.*)'

    state_match = re.search(state_pattern, email_text, re.IGNORECASE)
    date_range_match = re.search(date_range_pattern, email_text)
    columns_match = re.search(columns_pattern, email_text, re.IGNORECASE)

    state = state_match.group(1) if state_match else None
    start_date_str, end_date_str = date_range_match.groups() if date_range_match else (None, None)
    columns = [col.strip() for col in columns_match.group(1).split(',') if col.strip()] if columns_match else []

    try:
        start_date = parser.parse(start_date_str)
        end_date = parser.parse(end_date_str)
    except:
        start_date = None
        end_date = None

    return state, (start_date, end_date), columns

email = "Hi Alan, May I request a report for all completed transactions in Mississippi from 4/1/2022 through 3/31/2023, with the following columns: File number, property address, date completed, client name, product type, vendor first name, vendor last name, vendor license number, vendor license expiration date, date vendor first completed a report with Service 1st."

state, date_range, columns = extract_information(email)

print("State:", state)
print("Start Date:", date_range[0])
print("End Date:", date_range[1])
print("Columns:", columns)


State: Mississippi
Start Date: 2022-04-01 00:00:00
End Date: 2023-03-31 00:00:00
Columns: ['File number', 'property address', 'date completed', 'client name', 'product type', 'vendor first name', 'vendor last name', 'vendor license number', 'vendor license expiration date', 'date vendor first completed a report with Service 1st.']


In [1]:
import csv
import pandas as pd

# Read the CSV file, skipping rows with parsing errors
rows = []
with open('emails.csv', 'r', encoding='utf-8', errors='ignore') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cleaned_row = [cell.strip().replace('"', '') for cell in row]
        rows.append(cleaned_row)

# Get the column names from the first row and clean them
column_names = rows[0]
cleaned_column_names = [name.strip().replace('"', '') for name in column_names]

# Create a DataFrame excluding the first row (column names) to preserve cleaned column names
df = pd.DataFrame(rows[1:], columns=cleaned_column_names)

# Rename the first column as "Subject"
df = df.rename(columns={df.columns[0]: 'Subject'})

# Remove rows with empty body
df = df.dropna(subset=['Body'])

# Remove rows with Subject containing specific phrases or words
subject_patterns = ['Request received', 'Resolved Ticket', 'Automatic Reply', 'Your ticket has been created', 'Undeliverable',
                    'Email Delivery Failure', "Your message couldn't be delivered", 'Out of the Office']
subject_pattern_regex = '|'.join(subject_patterns)
df = df[~df['Subject'].str.contains(subject_pattern_regex, na=False, case=False, regex=True)]

# Remove rows with both "Ticket" and "Received" in the Subject
df = df[~(df['Subject'].str.contains('Ticket', case=False, na=False) & df['Subject'].str.contains('Received', case=False, na=False))]

# Remove rows with specific email address in the "From:(Address)" field
df = df[~df['From:(Address)'].str.contains('support@mercuryvmp.kayako.com', na=False, case=False)]

# Clean up the Body field
df['Body'] = df['Body'].str.replace('<.*?>', '', regex=True)  # Remove HTML tags
df['Body'] = df['Body'].str.replace('\n', ' ')  # Replace newline characters with spaces
df['Body'] = df['Body'].str.replace(' +', ' ', regex=True)  # Remove extra spaces

# Select the desired columns
desired_columns = ['Subject', 'Body', 'From:(Address)', 'To:(Address)']
df = df[desired_columns]

# Save the cleaned data to a new CSV file
df.to_csv('cleaned_emails.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'emails.csv'